# Using Weights & Biases Weave with AWS Bedrock

In this notebook you will learn to use our newly realeased tool for the LLM practitioner

You can use [Weave](https://wandb.github.io/weave/) to:

- Log and debug language model inputs, outputs, and traces
- Build rigorous, apples-to-apples evaluations for language model use cases
- Organize all the information generated across the LLM workflow, from experimentation to evaluations to production

## Setup

In [1]:
!pip install -Uq wandb weave


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import json
import boto3
from botocore.exceptions import ClientError

## log to your W&B account

In [ ]:
import wandb
wandb.login()

## create a W&B project to store your results

In [5]:
weave.init('bedrock-weave')

Logged in as W&B user capecape.
View Weave data at https://wandb.ai/capecape/bedrock-weave/weave


Decorate your function call, that's it!

In [15]:
@weave.op() # <- just add this 😎
def generate_text(
    model_id: str, 
    prompt: str, 
    max_tokens: int=400,
    temperature: float=0.7,
    top_p: float=0.7,
    top_k: int=50,
) -> dict:
    bedrock = boto3.client(service_name='bedrock-runtime')


    body = json.dumps({
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
    })
    
    response = bedrock.invoke_model(
        body=body,
        modelId=model_id
    )

    response_body = json.loads(response.get('body').read())

    outputs = response_body.get('outputs')
    
    return outputs

Let's first try using Mistral 7B

In [16]:
model_id = 'mistral.mistral-7b-instruct-v0:2'

prompt = """<s>[INST] In Bash, how do I list all text files in the current directory
  (excluding subdirectories) that have been modified in the last month? [/INST]"""

outputs = generate_text(model_id, prompt)

for index, output in enumerate(outputs):

    print(f"Output {index + 1}\n----------")
    print(f"Text:\n{output['text']}\n")
    print(f"Stop reason: {output['stop_reason']}\n")

🍩 https://wandb.ai/capecape/bedrock-weave/r/call/14db8c3d-218f-4044-a1f8-c7fc8ac8bb9d
Output 1
----------
Text:
 You can use the `find` command with `-type f`, `-mtime`, and `-print` options to list all text files (`-name "\*.txt"`) in the current directory (`.) that have been modified within the last month (`-mtime -30`):

```bash
find . -type f -name "*.txt" -mtime -30 -print
```

This command searches for files in the current directory (`.`) with the type of a regular file (`-type f`), having a `.txt` extension (`-name "*.txt"`), and modified within the last 30 days (`-mtime -30`). The `-print` option is used to print the file names to the standard output.

Stop reason: stop

